In [1]:
import os

import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
year, month = 2023, 3

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [6]:
y_pred.std()

6.247488852238703

In [7]:
df["ride_id"] = f"{year:04d}/{month:02d}_" + df.index.astype("str")
df["predictions"] = y_pred
df_result = df[["ride_id", "predictions"]].copy()
output_file = "duration_predictions.parquet"
df_result.to_parquet(output_file, engine="pyarrow", compression=None, index=False)

In [8]:
os.path.getsize('duration_predictions.parquet') >> 20

65